In [ ]:
# QuantBook Analysis Tool 
# For more information see [https://www.quantconnect.com/docs/research/overview]

In [ ]:
def getOptionDf(symbol, start_time, end_time):
    """
    Parameters
    ----------
    symbol: str
        ticker to get option data for
    start_time: datetime
        datetime(<year>, <month>, <day>, <hour>, <minute>) 
    end_time: datetime
        datetime(<year>, <month>, <day>, <hour>, <minute>)
        
    Returns
    -------
    Pandas DataFrame
        Options data across the chain
    """
    qb = QuantBook()
    equity = qb.AddEquity(symbol)
    option = qb.AddOption(symbol)
    df = qb.GetOptionHistory(equity.Symbol, start_time, end_time).GetAllData().reset_index().dropna()
    df = df.set_index('time')
    return df

def getOptionStrikes(option_df):
    """
    Parameters
    ----------
    option_df: Pandas DataFrame
        options data from getOptionDf method
        
    Returns
    -------
    list
        Options chain strike prices
    """
    return option_df['strike'].unique()

def getDataForStrike(option_df, strike, option_type):
    """
    Parameters
    ----------
    option_df: Pandas DataFrame
        options data from getOptionDf method
    strike: int
        strike price you want data for
    option_type: str
        'Call' or 'Put'
    
    Returns
    -------
    Pandas DataFrame
        Data for a certain strike price call or put
    """
    return option_df[(option_df['strike'] == strike) & (option_df['type'] == option_type)]

def plotOptionStrikeAndType(strike_df, strike, option_type, plot_col):
    """
    Parameters
    ----------
    strike_df: Pandas DataFrame
        options data for certain strike and type from getDataForStrike method
    strike: int
        strike price you want data for
    option_type: str
        'Call' or 'Put'
    plot_col: str
        column from option_df to plot
    
    Returns
    -------
    Matplotlib plot
        Plot for a certain strike call or put
    """
    strike_df[plot_col].plot()
    plt.title('{} {} {} | {} to {} | {}'.format(strike_df['symbol'].iloc[0].split(' ')[0],
                                             strike,
                                             option_type,
                                             strike_df.index[0], 
                                             strike_df.index[-1], 
                                             plot_col))

def plotAllStrikesAndTypes(option_df, plot_col):
    """
    Parameters
    ----------
    option_df: Pandas DataFrame
        options data from getOptionDf method
    plot_col: str
        column from option_df to plot
    
    Returns
    -------
    Matplotlib plots
        Plots for all strikes calls and puts
    """
    for strike in getOptionStrikes(option_df):
        for option_type in ['Call', 'Put']:
            strike_df = getDataForStrike(option_df, strike, option_type)
            if not strike_df.empty:
                plotOptionStrikeAndType(strike_df, strike, option_type, plot_col)
                plt.show()

In [ ]:
# Example with NVIDIA 9/30/2021 Investor Day

start_time =  datetime(2021, 9, 30, 9, 30) 
end_time = datetime(2021, 9, 30, 16, 0) 
df = getOptionDf('INTU', start_time, end_time)
df

In [ ]:
plotAllStrikesAndTypes(df, 'asklow')